In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
benign_data_path = '/content/drive/MyDrive/sample_data/benign_chunk_6.csv'
malicious_data_path = '/content/drive/MyDrive/sample_data/malicous_chunk_4.csv'

html 수집 - 최종

In [4]:
pip install fake-useragent

In [5]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import json
import random

In [ ]:
# 정상 데이터 html 수집 + 라벨링

# CSV 파일에서 URL 목록 로드
url_file = benign_data_path
urls_df = pd.read_csv(url_file)

# URL 컬럼에서 URL 리스트 생성
urls = urls_df['Domain'].tolist()

# UserAgent 객체 생성
ua = UserAgent()

# 세션 생성 및 헤더 설정
session = requests.Session()
session.headers.update({
    'User-Agent': ua.random,  # 랜덤으로 생성된 User-Agent 사용
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com',
    'Connection': 'keep-alive'
})

# HTML 크롤링 및 DOM 구조 추출 함수
def extract_dom_structure(soup):
    # 태그를 재귀적으로 파싱하여 계층적 구조를 생성
    def parse_tag(tag):
        # 각 태그의 이름, 속성, 자식 태그 정보를 포함하는 딕셔너리
        return {
            'tag': tag.name,
            'attributes': tag.attrs,  # 태그의 속성
            'text': tag.get_text(strip=True),  # 태그의 텍스트 내용
            'children': [parse_tag(child) for child in tag.find_all(True, recursive=False)]  # 자식 태그
        }
    return parse_tag(soup)

# URL에서 DOM 트리 구조 추출
def save_dom_structure_with_label(url, label):
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        soup = BeautifulSoup(response.text, 'html.parser')  # HTML 파싱

        # DOM 구조 추출
        dom_structure = extract_dom_structure(soup)

        # URL과 함께 DOM 구조 및 라벨을 딕셔너리로 저장
        print(f"{url} 의 HTML 데이터 수집 성공했습니다.")  # 성공 메시지 출력
        return {"url": url, "dom_structure": dom_structure, "label": label}

    except requests.exceptions.RequestException as e:
        print(f"URL {url} 수집 실패: {e}")
        return None  # 실패한 URL은 None 반환

# 모든 URL에 대해 DOM 구조 추출 후 저장
all_data = []  # 모든 URL의 데이터를 저장할 리스트
failed_urls = []  # 수집 실패한 URL 기록
label = 0  # 악성 데이터 라벨 (정상 데이터의 경우 0으로 설정)

for url in urls:
    data = save_dom_structure_with_label(url, label)
    if data and isinstance(data, dict):  # 데이터가 성공적으로 수집되었을 때
        all_data.append(data)
    else:  # 데이터 수집이 실패한 경우
        failed_urls.append(url)

# 모든 URL의 DOM 구조를 하나의 JSON 파일로 저장
output_file = 'labeled_benign_data.json'

with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print(f"모든 라벨링된 DOM 구조가 '{output_file}' 파일로 저장되었습니다.")

# 수집 실패한 URL을 CSV로 저장
if failed_urls:
    failed_df = pd.DataFrame(failed_urls, columns=['failed_url'])
    failed_df.to_csv('failed_benign_urls.csv', index=False)
    print("수집 실패한 URL이 'failed_benign_urls.csv'로 저장되었습니다.")
else:
    print("모든 URL이 성공적으로 처리되었습니다.")


https://apnews.com 의 HTML 데이터 수집 성공했습니다.
https://nypost.com 의 HTML 데이터 수집 성공했습니다.
https://store.steampowered.com 의 HTML 데이터 수집 성공했습니다.
https://giphy.com 의 HTML 데이터 수집 성공했습니다.
https://news.google.com 의 HTML 데이터 수집 성공했습니다.
https://digg.com 의 HTML 데이터 수집 성공했습니다.
https://jhu.edu 의 HTML 데이터 수집 성공했습니다.
https://disqus.com 의 HTML 데이터 수집 성공했습니다.
URL https://perfectdomain.com 수집 실패: 403 Client Error: Forbidden for url: https://perfectdomain.com/
https://postgresql.org 의 HTML 데이터 수집 성공했습니다.
https://walmart.com 의 HTML 데이터 수집 성공했습니다.
https://senate.gov 의 HTML 데이터 수집 성공했습니다.
https://dropcatch.com 의 HTML 데이터 수집 성공했습니다.
https://naver.com 의 HTML 데이터 수집 성공했습니다.
https://yahoo.co.jp 의 HTML 데이터 수집 성공했습니다.
https://telegra.ph 의 HTML 데이터 수집 성공했습니다.
URL https://chatgpt.com 수집 실패: 403 Client Error: Forbidden for url: https://chatgpt.com/
https://ucsd.edu 의 HTML 데이터 수집 성공했습니다.
https://ok.ru 의 HTML 데이터 수집 성공했습니다.
https://letsencrypt.org 의 HTML 데이터 수집 성공했습니다.
https://espn.com 의 HTML 데이터 수집 성공했습니다.
https://fedorapr

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Streaming output truncated to the last 5000 lines.
URL http://toptal.com 수집 실패: 403 Client Error: Forbidden for url: https://www.toptal.com/
URL http://wikispaces.com 수집 실패: HTTPConnectionPool(host='wikispaces.com', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7ed12cae6200>: Failed to resolve 'wikispaces.com' ([Errno -5] No address associated with hostname)"))
http://buymeacoffee.com 의 HTML 데이터 수집 성공했습니다.
http://acast.com 의 HTML 데이터 수집 성공했습니다.
URL http://fsf.org 수집 실패: HTTPConnectionPool(host='fsf.org', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ed14216cac0>, 'Connection to fsf.org timed out. (connect timeout=10)'))
http://hackernoon.com 의 HTML 데이터 수집 성공했습니다.
http://wolframalpha.com 의 HTML 데이터 수집 성공했습니다.
http://bayern.de 의 HTML 데이터 수집 성공했습니다.
http://uw.edu 의 HTML 데이터 수집 성공했습니다.
http://mathworks.com 의 HTML 데이터 수집 성공했습니다.
URL http://m

In [ ]:
# 악성 데이터 html 수집 + 라벨링

# CSV 파일에서 URL 목록 로드
url_file = malicious_data_path
urls_df = pd.read_csv(url_file)

# URL 컬럼에서 URL 리스트 생성
urls = urls_df['url'].tolist()

# UserAgent 객체 생성
ua = UserAgent()

# 세션 생성 및 헤더 설정
session = requests.Session()
session.headers.update({
    'User-Agent': ua.random,  # 랜덤으로 생성된 User-Agent 사용
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com',
    'Connection': 'keep-alive'
})

# HTML 크롤링 및 DOM 구조 추출 함수
def extract_dom_structure(soup):
    # 태그를 재귀적으로 파싱하여 계층적 구조를 생성
    def parse_tag(tag):
        # 각 태그의 이름, 속성, 자식 태그 정보를 포함하는 딕셔너리
        return {
            'tag': tag.name,
            'attributes': tag.attrs,  # 태그의 속성
            'text': tag.get_text(strip=True),  # 태그의 텍스트 내용
            'children': [parse_tag(child) for child in tag.find_all(True, recursive=False)]  # 자식 태그
        }
    return parse_tag(soup)

# URL에서 DOM 트리 구조 추출
def save_dom_structure_with_label(url, label):
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        soup = BeautifulSoup(response.text, 'html.parser')  # HTML 파싱

        # DOM 구조 추출
        dom_structure = extract_dom_structure(soup)

        # URL과 함께 DOM 구조 및 라벨을 딕셔너리로 저장
        print(f"{url} 의 HTML 데이터 수집 성공했습니다.")  # 성공 메시지 출력
        return {"url": url, "dom_structure": dom_structure, "label": label}

    except requests.exceptions.RequestException as e:
        print(f"URL {url} 수집 실패: {e}")
        return None  # 실패한 URL은 None 반환

# 모든 URL에 대해 DOM 구조 추출 후 저장
all_data = []  # 모든 URL의 데이터를 저장할 리스트
failed_urls = []  # 수집 실패한 URL 기록
label = 1  # 악성 데이터 라벨 (정상 데이터의 경우 0으로 설정)

for url in urls:
    data = save_dom_structure_with_label(url, label)
    if data and isinstance(data, dict):  # 데이터가 성공적으로 수집되었을 때
        all_data.append(data)
    else:  # 데이터 수집이 실패한 경우
        failed_urls.append(url)

# 모든 URL의 DOM 구조를 하나의 JSON 파일로 저장
output_file = 'labeled_malicious_data.json'

with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print(f"모든 라벨링된 DOM 구조가 '{output_file}' 파일로 저장되었습니다.")

# 수집 실패한 URL을 CSV로 저장
if failed_urls:
    failed_df = pd.DataFrame(failed_urls, columns=['failed_url'])
    failed_df.to_csv('failed_malicious_urls.csv', index=False)
    print("수집 실패한 URL이 'failed_malicious_urls.csv'로 저장되었습니다.")
else:
    print("모든 URL이 성공적으로 처리되었습니다.")


https://usps.com-trackqnj.top/track/ 의 HTML 데이터 수집 성공했습니다.
http://bafkreigdalhw5kpc2r3hzjyzzk555p62ehbqca2jf3nkz2f3glsjdomkjy.ipfs.flk-ipfs.xyz/ 의 HTML 데이터 수집 성공했습니다.
https://coi4.vercel.app/verset.html 의 HTML 데이터 수집 성공했습니다.
http://rebrand.ly/2nuks6o 의 HTML 데이터 수집 성공했습니다.
http://airdnd-duynd1006.vercel.app/ 의 HTML 데이터 수집 성공했습니다.
https://dovacare.com/mex.html 의 HTML 데이터 수집 성공했습니다.
http://trimmer.to/YNWTy 의 HTML 데이터 수집 성공했습니다.
https://sgwlfugw.work/ 의 HTML 데이터 수집 성공했습니다.
http://mail.8724319-coinbase.com/review..................................................................................... 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/2647f06/Sign_in.php 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/d32acdd/ 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/5d0c10f/Sign_in.php 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/739f3fc/Sign_in.php 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/0e5ffcd/ 의 HTML 데이터 수집 성공했습니다.
https://webausin.com/v/NRD/b335acb/Sign_in.php 의 HTML 데이

In [ ]:
# 정상 + 악성 데이터 병합

# 정상 데이터 로드
with open('labeled_benign_data.json', 'r', encoding='utf-8') as benign_file:
    benign_data = json.load(benign_file)

# 악성 데이터 로드
with open('labeled_malicious_data.json', 'r', encoding='utf-8') as malicious_file:
    malicious_data = json.load(malicious_file)

# 병합
combined_data = benign_data + malicious_data
random.shuffle(combined_data)  # 데이터를 랜덤하게 섞기

# 병합된 데이터 저장
with open('combined_data.json', 'w', encoding='utf-8') as output_file:
    json.dump(combined_data, output_file, ensure_ascii=False, indent=4)

print(f"정상 데이터와 악성 데이터가 병합되어 'combined_data.json'에 저장되었습니다.")


정상 데이터와 악성 데이터가 병합되어 'combined_data.json'에 저장되었습니다.


In [ ]:
# 데이터셋 분리
train_ratio = 0.7
val_ratio = 0.15

train_size = int(len(combined_data) * train_ratio)
val_size = int(len(combined_data) * val_ratio)

train_data = combined_data[:train_size]
val_data = combined_data[train_size:train_size + val_size]
test_data = combined_data[train_size + val_size:]

# 각 데이터셋 저장
with open('train_data.json', 'w', encoding='utf-8') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

with open('val_data.json', 'w', encoding='utf-8') as val_file:
    json.dump(val_data, val_file, ensure_ascii=False, indent=4)

with open('test_data.json', 'w', encoding='utf-8') as test_file:
    json.dump(test_data, test_file, ensure_ascii=False, indent=4)

print("데이터셋이 학습(train), 검증(validation), 테스트(test)로 분리되었습니다.")

데이터셋이 학습(train), 검증(validation), 테스트(test)로 분리되었습니다.


GCN 모델 - 작업중

In [ ]:
!pip install torch
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=538974 sha256=d97a327c11f47975609208c5c26cd9a290af8d9814bb51ffc9c1313c448fd07d
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1104576 sha256=0f74194b024834282c814a924dcdcc72b3d3e06

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import json

# JSON 데이터 로드 함수
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 그래프 생성 함수
def create_graph(data, tag_to_idx):
    node_features = []
    edges_src = []
    edges_dst = []
    labels = []
    node_idx_map = {}
    node_counter = 0

    for item in data:
        dom_structure = item['dom_structure']
        label = item['label']

        queue = [(dom_structure, None)]
        while queue:
            current_node, parent_idx = queue.pop(0)
            node_idx = node_counter
            node_idx_map[id(current_node)] = node_idx
            node_features.append(current_node['tag'])
            labels.append(label)  # 노드별 레이블 생성
            node_counter += 1

            if parent_idx is not None:
                edges_src.append(parent_idx)
                edges_dst.append(node_idx)

            for child in current_node['children']:
                queue.append((child, node_idx))

    # 노드 특징 벡터화 (원-핫 인코딩 방식)
    node_features_tensor = torch.zeros(len(node_features), len(tag_to_idx), dtype=torch.float)
    for i, tag in enumerate(node_features):
        node_features_tensor[i, tag_to_idx[tag]] = 1.0

    # 레이블 텐서 생성
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    # PyG 데이터 생성
    edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
    return Data(x=node_features_tensor, edge_index=edge_index, y=labels_tensor)


# 데이터 로드
train_data = load_data('train_data.json')
val_data = load_data('val_data.json')
test_data = load_data('test_data.json')

# 고유 태그 수집 및 매핑 생성
unique_tags = set()
for graph_data in [train_data, val_data, test_data]:
    for item in graph_data:
        queue = [item['dom_structure']]
        while queue:
            current_node = queue.pop(0)
            unique_tags.add(current_node['tag'])
            queue.extend(current_node['children'])

tag_to_idx = {tag: idx for idx, tag in enumerate(unique_tags)}

# 그래프 생성
train_graph = create_graph(train_data, tag_to_idx)
val_graph = create_graph(val_data, tag_to_idx)
test_graph = create_graph(test_data, tag_to_idx)

# GCN 모델 정의
class GCNModel(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(in_feats, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.conv3 = GCNConv(hidden_size, out_feats)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        h = self.conv1(x, edge_index)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.conv2(h, edge_index)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.conv3(h, edge_index)
        return h # [노드 수, 클래스 수]


# 모델 초기화
in_feats = train_graph.x.shape[1]  # 입력 특징 크기
hidden_size = 16
out_feats = 2  # 클래스 수 (정상/악성)
model = GCNModel(in_feats, hidden_size, out_feats)

# 학습 준비
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# 학습
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(train_graph)
    loss = loss_fn(logits, train_graph.y)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# 검증
model.eval()
with torch.no_grad():
    val_logits = model(val_graph)
    val_predictions = torch.argmax(val_logits, dim=1)
    correct = (val_predictions == val_graph.y).sum().item()
    accuracy = correct / len(val_graph.y)
    print(f"Validation Accuracy: {accuracy:.4f}")